# Kita mulai praktek dari K-Fold Cross-Validation

# 🛠 Contoh: K-Fold CV untuk Regresi

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Inisialisasi model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# K-Fold Cross-Validation (misal, K=5)
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Cetak hasil
print(f"Accuracy per fold: {scores}")
print(f"Mean Accuracy: {scores.mean():.4f}")


Accuracy per fold: [0.96666667 0.96666667 0.93333333 0.96666667 1.        ]
Mean Accuracy: 0.9667


## 🛠 Contoh: K-Fold CV untuk Regresi
#### Kita pakai dataset diabetes dari sklearn.datasets dan model Random Forest Regressor.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_diabetes
from sklearn.metrics import make_scorer, mean_squared_error

# Load dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Inisialisasi model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Definisikan scoring metric (pakai negatif MSE karena cross_val_score meminimalkan skor)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# K-Fold Cross-Validation (misal, K=5)
scores = cross_val_score(model, X, y, cv=5, scoring=mse_scorer)

# Cetak hasil
print(f"MSE per fold: {scores}")
print(f"Mean MSE: {scores.mean():.4f}")

import numpy as np

rmse_scores = np.sqrt(-scores)
print(f"RMSE per fold: {rmse_scores}")
print(f"Mean RMSE: {rmse_scores.mean():.4f}")

# 🔎 Analisis Hasil
# 1️⃣ RMSE lebih interpretatif dibanding MSE karena skalanya sama dengan target aslinya.
# 2️⃣ Variasi antar fold cukup kecil, berarti model kita cukup konsisten.
# 3️⃣ Kalau mau hasil lebih bagus, bisa coba hyperparameter tuning atau feature engineering.


MSE per fold: [-3006.94880562 -3065.96508315 -3571.39149545 -3405.32815227
 -3804.19287614]
Mean MSE: -3370.7653
RMSE per fold: [54.83565269 55.3711575  59.76112027 58.35518959 61.67813937]
Mean RMSE: 58.0003


## Stratified K-Fold

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Inisialisasi model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Inisialisasi Stratified K-Fold (K=5)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluasi model dengan Stratified K-Fold
scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')

# Cetak hasil
print(f"Accuracy per fold: {scores}")
print(f"Mean Accuracy: {scores.mean():.4f}")

# 🔎 Analisis Hasil
# 1️⃣ Variasi antar fold kecil → Model kita stabil dan tidak overfitting ke satu fold tertentu.
# 2️⃣ Akurasi tinggi → Random Forest cocok untuk dataset Iris, yang relatif seimbang dan tidak terlalu kompleks.
# 3️⃣ Stratified K-Fold membantu memastikan distribusi kelas tetap seimbang di setiap fold, sehingga model lebih adil dalam belajar.

Accuracy per fold: [0.96666667 0.96666667 0.93333333 0.96666667 0.9       ]
Mean Accuracy: 0.9467


## Stratified K-Fold untuk regresi

#### 🔹 Bagaimana Cara Adaptasi Stratified K-Fold ke Regresi?
#### 1️⃣ Binning target variable → Ubah nilai target kontinu menjadi kategori (misal, bagi jadi 5 kelompok berdasarkan kuantil).
#### 2️⃣ Gunakan Stratified K-Fold dengan kategori tersebut untuk menjaga distribusi target tetap seimbang di tiap fold.

In [6]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Load dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Konversi target kontinu menjadi kategori dengan binning (misal, 5 bins)
y_binned = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile').fit_transform(y.reshape(-1,1)).ravel()

# Inisialisasi model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Inisialisasi Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definisikan scoring metric (pakai negatif MSE karena cross_val_score meminimalkan skor)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Evaluasi model
scores = cross_val_score(model, X, y, cv=skf, scoring=mse_scorer)

# Cetak hasil
print(f"MSE per fold: {scores}")
print(f"Mean MSE: {scores.mean():.4f}")

# 🔎 Analisis Hasil
# ⚠ Warning: "Least populated class in y has only 1 member"
# 🔹 Artinya, saat kita membagi target ke dalam 5 kategori, ada satu kategori yang cuma punya 1 data point, jadi sulit untuk dibagi ke dalam 5 fold.
# 🔹 Solusi?
# ✅ Kurangi jumlah bins jadi misalnya 3 atau 4 (karena datasetnya kecil).
# ✅ Pakai metode K-Fold biasa kalau datasetnya tidak cocok untuk binning.


c:\Users\formylife\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


MSE per fold: [-3235.2827382  -3015.34030449 -4041.1152625  -3844.60083523
 -2950.03258864]
Mean MSE: -3417.2743


# Time Series Split

##  🔹 Apa Itu Time Series Split?
#### 🔹 Berbeda dengan K-Fold biasa, karena data dibagi secara berurutan (bukan acak).
#### 🔹 Digunakan untuk data berbasis waktu, misalnya prediksi harga saham, suhu, atau penjualan harian.
#### 🔹 Setiap fold lebih besar dari fold sebelumnya, karena model tidak boleh "melihat masa depan".

## 🔹 Time Series Split untuk Klasifikasi
#### Meskipun Time Series Split lebih sering dipakai di regresi (misal, harga saham, cuaca, dll.), kita bisa pakai untuk klasifikasi berbasis waktu, misalnya:
#### ✅ Prediksi churn pelanggan (berdasarkan aktivitas sebelumnya).
#### ✅ Prediksi kredit macet (berdasarkan riwayat transaksi).
#### ✅ Deteksi penipuan (fraud detection).

In [8]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, make_scorer
import numpy as np

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Inisialisasi model (Decision Tree lebih toleran terhadap class imbalance)
model = DecisionTreeClassifier()

# Inisialisasi Time Series Split (K=5)
tscv = TimeSeriesSplit(n_splits=5)

# Definisikan scoring metric (Accuracy)
accuracy_scorer = make_scorer(accuracy_score)

# Evaluasi model dengan Time Series Split
scores = cross_val_score(model, X, y, cv=tscv, scoring=accuracy_scorer)

# Cetak hasil
print(f"Accuracy per fold: {scores}")
print(f"Mean Accuracy: {scores.mean():.4f}")


Accuracy per fold: [1.  0.  1.  0.  0.8]
Mean Accuracy: 0.5600


## Time Series Split untuk regresi
#### 🔹 Langkah-langkah
#### 1️⃣ Gunakan dataset time-series, misalnya dataset Diabetes dari Scikit-Learn.
#### 2️⃣ Bagi data dengan TimeSeriesSplit (tanpa shuffle).
#### 3️⃣ Latih model regresi (Linear Regression).
#### 4️⃣ Evaluasi performa dengan MSE dan RMSE.

In [10]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np

# Load dataset Diabetes (cocok untuk regresi)
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Inisialisasi model Linear Regression
model = LinearRegression()

# Inisialisasi Time Series Split (5 fold)
tscv = TimeSeriesSplit(n_splits=5)

# Definisikan scoring metric (MSE)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Evaluasi model dengan Time Series Split
mse_scores = cross_val_score(model, X, y, cv=tscv, scoring=mse_scorer)

# Hitung RMSE
rmse_scores = np.sqrt(-mse_scores)

# Cetak hasil
print(f"MSE per fold: {mse_scores}")
print(f"Mean MSE: {mse_scores.mean():.4f}")
print(f"RMSE per fold: {rmse_scores}")
print(f"Mean RMSE: {rmse_scores.mean():.4f}")

# 📊 Analisis Hasil
# 🔹 MSE per Fold → Skor semakin kecil di fold terakhir (-2459.87), menunjukkan error lebih kecil seiring bertambahnya data.
# 🔹 RMSE per Fold → Turun dari 62.16 ke 49.59, artinya model makin baik dalam memprediksi pada fold terakhir.
# 🔹 Mean RMSE = 56.87 → Model cukup stabil, tapi masih bisa ditingkatkan.


MSE per fold: [-3865.02037348 -3656.75210602 -3192.21541771 -3091.5807308
 -2459.86963436]
Mean MSE: -3253.0877
RMSE per fold: [62.16928802 60.47108488 56.49969396 55.60198495 49.59707284]
Mean RMSE: 56.8678


## Leave-One-Out Cross-Validation (LOOCV).
## 🔹 Kita Mulai dengan Regresi Dulu

In [12]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error
import numpy as np

# Load dataset Diabetes
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Inisialisasi model
model = LinearRegression()

# Inisialisasi LOOCV
loo = LeaveOneOut()

# Simpan hasil MSE
mse_scores = []

# Looping untuk setiap iterasi LOOCV
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Latih model
    model.fit(X_train, y_train)
    
    # Prediksi
    y_pred = model.predict(X_test)
    
    # Hitung MSE
    mse_scores.append(mean_squared_error(y_test, y_pred))

# Hitung rata-rata MSE & RMSE
mean_mse = np.mean(mse_scores)
mean_rmse = np.sqrt(mean_mse)

# Cetak hasil
print(f"Mean MSE: {mean_mse:.4f}")
print(f"Mean RMSE: {mean_rmse:.4f}")

# 📊 Analisis Hasil LOOCV
# 🔹 Mean MSE: 3001.75 → Lebih rendah dibanding Time Series Split (3253.09), yang berarti model lebih stabil.
# 🔹 Mean RMSE: 54.79 → Sedikit lebih kecil dibanding sebelumnya (56.86), menunjukkan prediksi sedikit lebih akurat.


Mean MSE: 3001.7528
Mean RMSE: 54.7883



# 📌 Kesimpulan:
#### ✅ LOOCV memanfaatkan hampir seluruh data untuk training, jadi hasilnya lebih stabil.
#### ✅ Tetapi, prosesnya lebih lama & computationally expensive.
#### ✅ Model ini masih bisa ditingkatkan dengan Feature Engineering atau Model Tuning.


## Nested Cross-Validation
## 🔹 Implementasi Nested CV untuk Regresi

In [16]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.datasets import load_diabetes
import numpy as np

# Load dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Outer Loop: 5-Fold Cross-Validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Ridge Regression dengan GridSearchCV di Inner Loop
param_grid = {"alpha": [0.1, 1, 10, 100]}  # Hyperparameter Ridge
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)  # Inner Loop

# Model dengan GridSearchCV
model = GridSearchCV(Ridge(), param_grid, cv=inner_cv, scoring='neg_mean_squared_error')

# Evaluasi Nested Cross-Validation
nested_scores = cross_val_score(model, X, y, cv=outer_cv, scoring='neg_mean_squared_error')

# Hitung rata-rata MSE & RMSE
mean_mse = -np.mean(nested_scores)
mean_rmse = np.sqrt(mean_mse)

# Cetak hasil
print(f"Mean MSE (Nested CV): {mean_mse:.4f}")
print(f"Mean RMSE (Nested CV): {mean_rmse:.4f}")

# 🔥 Nice! Hasilnya udah keluar!
# 📊 Analisis Hasil Nested Cross-Validation:
# 🔹 Mean MSE: 3013.81 → Hampir sama dengan LOOCV (3001.75), yang berarti model cukup stabil.
# 🔹 Mean RMSE: 54.90 → Sedikit lebih tinggi dari LOOCV (54.79), tapi perbedaannya kecil.

Mean MSE (Nested CV): 3013.8106
Mean RMSE (Nested CV): 54.8982


## 📌 Kesimpulan:
#### ✅ Nested CV memberikan estimasi performa yang lebih akurat karena sudah menghindari data leakage dari tuning hyperparameter.
#### ✅ Performanya mirip dengan LOOCV, tapi Nested CV lebih efisien dalam pemrosesan dibanding LOOCV yang sangat berat.
#### ✅ Model Ridge Regression sudah cukup baik, tapi masih bisa ditingkatkan dengan tuning lebih lanjut (misalnya pakai RandomizedSearchCV atau Bayesian Optimization).

## 🔹 Implementasi Nested Cross-Validation untuk Klasifikasi

In [18]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import numpy as np

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Outer Loop: 5-Fold Cross-Validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Hyperparameter tuning dengan GridSearchCV di Inner Loop
param_grid = {"C": [0.01, 0.1, 1, 10, 100]}  # Regularization parameter
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Model dengan GridSearchCV
model = GridSearchCV(LogisticRegression(max_iter=2000), param_grid, cv=inner_cv, scoring='accuracy')

# Evaluasi Nested Cross-Validation
nested_scores = cross_val_score(model, X, y, cv=outer_cv, scoring='accuracy')

# Hitung rata-rata akurasi
mean_accuracy = np.mean(nested_scores)

# Cetak hasil
print(f"Mean Accuracy (Nested CV): {mean_accuracy:.4f}")

# 📊 Analisis Hasil:
# 🔹 Mean Accuracy (Nested CV): 0.9733 (97.33%) → Model Logistic Regression dengan GridSearchCV bekerja sangat baik untuk dataset Iris.
# 🔹 Tuning Hyperparameter dengan Nested CV → Mengurangi risiko data leakage, sehingga estimasi performa lebih realistis.


Mean Accuracy (Nested CV): 0.9733


## 📌 Kesimpulan:
#### ✅ Model kita udah optimal → Akurasi 97.33% itu udah sangat bagus buat dataset ini.
#### ✅ Nested CV validasi hyperparameter dengan aman, karena memisahkan training-validation di dalam inner loop, lalu evaluasi final di outer loop.
#### ✅ Tuning dengan GridSearchCV efektif, tapi bisa dicoba RandomizedSearchCV atau Optuna untuk eksperimen lebih luas.